In [1]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
#################################################################################
import datetime as dt
import xarray as xr
#from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
#import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sat_data/'
dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
#output files
#filename_out_nc='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.nc'
#filename_out_kml='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.kml'
#################################################################################
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.mlab as mlab
palette = copy(plt.cm.jet)
palette.set_over('r', 1.0)
palette.set_under('g', 1.0)
palette.set_bad(alpha = 0.0)

In [2]:
#read in target lat/lon (ccmp array)
ccmp_filename='F:/data/sat_data/ccmp/v02.0/Y2003/M10/CCMP_Wind_Analysis_20031009_V02.0_L3.0_RSS.nc'
ds_target=xr.open_dataset(ccmp_filename)  #don't need nobs
ds_target_lon=ds_target.longitude
ds_target_lat=ds_target.latitude


In [3]:
dir_ccmp='F:/data/sat_data/ccmp/v02.0/Y'
date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
dx=0.25
dy=0.25
dx_offset = -179.875
dy_offset = 78.3750
for root, dirs, files in os.walk(dir_in, topdown=False):
    #print(files)
    #for ii in range(40,90): 
    #print(root[31:35])
    for name in files:
        istart_storm=0
        #name = files[ii]
        fname_in=os.path.join(root, name)
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat[0,:]
        lons = dsx.lon[0,:]
        dysince = dsx.time
        icycle=0
        minlon=min(lons.values)-7
        maxlon=max(lons.values)+7
        minlat=min(lats.values)-7
        maxlat=max(lats.values)+7
        if minlon<10. and maxlon>350.:  #wrapping around meridion need to cal new min/max lon
            minlon=max(lons[lons<180].values)+7
            maxlon=min(lons[lons>180].values)-7
            icycle=1 #set flag for wraparound
        dims=lats.shape
        print(dims)
        tdim=dims[0]
        tem_date=[0]*tdim #print(dysince.values)
        for i in range(0,tdim):
            tem_date[i]=date_1858+dt.timedelta(days=float(dysince[0,i].values))  #create new time array that can be queried for year etc
        minjdy = min(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        minyear =min(tem_date).year #create new time array that can be queried for year etc
        maxjdy = max(tem_date).timetuple().tm_yday  #create new time array that can be queried for year etc
        maxyear =max(tem_date).year  #create new time array that can be queried for year etc
        dif = max(tem_date)-min(tem_date)
        tdim=int(dif.days)  
        print(minjdy,maxjdy,minlon,maxlon,minlat,maxlat)
        ds_new=[]
        new_coord=[]
        for incr_dy in range(-5,tdim+25): #from -5 to 25 days after
            storm_date = tem_date[0]+dt.timedelta(days=incr_dy)            
            #print(storm_date)
            syr=str(storm_date.year)
            smon=str(storm_date.month)
            sdym=str(storm_date.day)
            sjdy=str(storm_date.timetuple().tm_yday)
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
            mur_filename = dir_mur + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            print(mur_filename)
            ds=xr.open_dataset(mur_filename,drop_variables=('analysis_error','sea_ice_fraction'))  #don't need nobs
            ds_rolled = ds.assign_coords(lon=(ds.lon % 360)).roll(lon=(ds.dims['lon'] // 2))
            sst=ds_rolled.analysed_sst
            #interpolate 1 km mur data onto CCMP grid
            sst_interp = sst.interp(lat=ds_target_lat,lon=ds_target_lon)
            #select lat/lon region
            if icycle==0:
                ds2=sst_interp.sel(latitude=slice(minlat,maxlat),longitude=slice(minlon,maxlon))
            else:
                ds2=sst_interp.sel(latitude=slice(minlat,maxlat),longitude=(sst_interp.longitude < minlon) | (sst_interp.longitude > maxlon))
            #print(ds2)
            ds_new.append(ds2)
            new_coord.append(storm_date)
        combined = xr.concat(ds_new, dim='time')
        combined.coords['time'] = new_coord
        fname_out=dir_out + 'mur' + root[31:35] + name[0:3] + '.nc'
        print(fname_out)
        combined.to_netcdf(fname_out)
        #break
    #break
#print(ds_new)


001annual_storm_info.nc f:/data/tc_wakes/database/info/2004\001annual_storm_info.nc
(26,)
271 277 56.299998585134745 74.3999984934926 -13.999999843537807 2.8000000938773155
F:/data/sst/jpl_mur/v4.1/2003/266/20030923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/267/20030924090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/268/20030925090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/269/20030926090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/270/20030927090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/271/20030928090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/272/20030929090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/273/20030930090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/274/20031001090000-JPL-L4_GHRSST-SSTf

F:/data/sst/jpl_mur/v4.1/2003/334/20031130090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/335/20031201090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/336/20031202090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/337/20031203090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/338/20031204090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/339/20031205090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/340/20031206090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/341/20031207090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/342/20031208090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/343/20031209090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/344/20031210090000-J

F:/data/sst/jpl_mur/v4.1/2003/346/20031212090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/347/20031213090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/348/20031214090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/349/20031215090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/350/20031216090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/351/20031217090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/352/20031218090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/353/20031219090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/354/20031220090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/355/20031221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/356/20031222090000-J

F:/data/sst/jpl_mur/v4.1/2003/362/20031228090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/363/20031229090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/364/20031230090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2003/365/20031231090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/001/20040101090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/002/20040102090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/003/20040103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/004/20040104090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/005/20040105090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/006/20040106090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/007/20040107090000-J

F:/data/sst/jpl_mur/v4.1/2004/064/20040304090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/065/20040305090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/066/20040306090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/067/20040307090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
f:/data/tc_wakes/database/sat_data/mur2004008.nc
009annual_storm_info.nc f:/data/tc_wakes/database/info/2004\009annual_storm_info.nc
(54,)
26 39 54.599998623132706 96.99999798834324 -38.199999302625656 -3.8999997563660145
F:/data/sst/jpl_mur/v4.1/2004/021/20040121090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/022/20040122090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/023/20040123090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/024/20040124090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mu

...

F:/data/sst/jpl_mur/v4.1/2004/070/20040310090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/071/20040311090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/072/20040312090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/073/20040313090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/074/20040314090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/075/20040315090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/076/20040316090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/077/20040317090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/078/20040318090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/079/20040319090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/080/20040320090000-J

F:/data/sst/jpl_mur/v4.1/2004/081/20040321090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/082/20040322090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/083/20040323090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/084/20040324090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/085/20040325090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/086/20040326090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/087/20040327090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/088/20040328090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/089/20040329090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/090/20040330090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/091/20040331090000-J

F:/data/sst/jpl_mur/v4.1/2004/102/20040411090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/103/20040412090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/104/20040413090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/105/20040414090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/106/20040415090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/107/20040416090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/108/20040417090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/109/20040418090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/110/20040419090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/111/20040420090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/112/20040421090000-J

F:/data/sst/jpl_mur/v4.1/2004/137/20040516090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/138/20040517090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/139/20040518090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/140/20040519090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/141/20040520090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/142/20040521090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/143/20040522090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/144/20040523090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/145/20040524090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/146/20040525090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
F:/data/sst/jpl_mur/v4.1/2004/147/20040526090000-J

KeyboardInterrupt: 

<xarray.Dataset>
Dimensions:       (lat: 17999, lon: 36000, time: 1)
Coordinates:
  * time          (time) datetime64[ns] 2003-09-23T09:00:00
  * lat           (lat) float32 -89.99 -89.98 -89.97 -89.96 -89.95 -89.94 ...
  * lon           (lon) float32 -179.99 -179.98 -179.97 -179.96 -179.95 ...
Data variables:
    analysed_sst  (time, lat, lon) float32 ...
    mask          (time, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.5
    title:                      Daily MUR SST, Final product
    summary:                    A merged, multi-sensor L4 Foundation SST anal...
    references:                 http://podaac.jpl.nasa.gov/Multi-scale_Ultra-...
    institution:                Jet Propulsion Laboratory
    history:                    created at nominal 4-day latency; replaced nr...
    comment:                    MUR = "Multi-scale Ultra-high Reolution"
    license:                    These data are available free of charge under...
    id:                    

OSError: [Errno -101] NetCDF: HDF error: b'f:\\data\\model_data\\gmao\\flux\\MERRA2_300.tavg1_2d_flx_Nx.20041114.SUB.nc4'